Imports

In [2]:
import pandas as pd
import numpy as np

Preprocessing Data

In [ ]:
sales = pd.read_csv("raw_data/sales.csv")
item_price_cost = pd.read_csv('raw_data/ItemPriceCost.csv') # Product.csv
stores = pd.read_csv('raw_data/stores.csv')
inventory = pd.read_csv('raw_data/inventory.csv')

sales.head(2)

,Sale_ID,Date,Store_ID,Product_ID,Units
0,1,2022-01-01,24,4,1
1,2,2022-01-01,28,1,1


In [8]:
item_price_cost.head(2)

,Product_ID,Year,Quarter,Product_Name,Product_Category,Product_Cost,Product_Price,Date,Month,Actual_Product_Price
0,1,2022,1,Action Figure,Toys,9.99,15.99,2022-01-01,1,15.83
1,1,2022,1,Action Figure,Toys,9.99,15.99,2022-01-02,1,14.98


In [9]:
stores.head(2)

,Store_ID,Store_Name,Store_City,Store_Location,Store_Open_Date
0,1,Maven Toys Guadalajara 1,Guadalajara,Residential,1992-09-18
1,2,Maven Toys Monterrey 1,Monterrey,Residential,1995-04-27


In [10]:
inventory.head(2)

,Store_ID,Product_ID,Stock_On_Hand
0,1,1,27
1,1,2,0


In [5]:
merged_data = sales.merge(item_price_cost,how='left',left_on=['Product_ID','Date'],right_on=['Product_ID','Date']).merge(stores,how='left',on='Store_ID')
merged_data.head()

,Sale_ID,Date,Store_ID,Product_ID,Units,Year,Quarter,Product_Name,Product_Category,Product_Cost,Product_Price,Month,Actual_Product_Price,Store_Name,Store_City,Store_Location,Store_Open_Date
0,1,2022-01-01,24,4,1,2022,1,Chutes & Ladders,Games,9.99,12.99,1,12.39,Maven Toys Aguascalientes 1,Aguascalientes,Downtown,2010-07-31
1,2,2022-01-01,28,1,1,2022,1,Action Figure,Toys,9.99,15.99,1,15.83,Maven Toys Puebla 2,Puebla,Downtown,2011-04-01
2,3,2022-01-01,6,8,1,2022,1,Deck Of Cards,Games,3.99,6.99,1,6.74,Maven Toys Mexicali 1,Mexicali,Commercial,2003-12-13
3,4,2022-01-01,48,7,1,2022,1,Dart Gun,Sports & Outdoors,11.99,15.99,1,14.97,Maven Toys Saltillo 2,Saltillo,Commercial,2016-03-23
4,5,2022-01-01,44,18,1,2022,1,Lego Bricks,Toys,34.99,38.99,1,38.73,Maven Toys Puebla 3,Puebla,Residential,2014-12-27


In [13]:
merged_data.isnull().sum()

Sale_ID                 0
Date                    0
Store_ID                0
Product_ID              0
Units                   0
Year                    0
Quarter                 0
Product_Name            0
Product_Category        0
Product_Cost            0
Product_Price           0
Month                   0
Actual_Product_Price    0
Store_Name              0
Store_City              0
Store_Location          0
Store_Open_Date         0
Revenue                 0
xRevenue                0
Profit                  0
dtype: int64

In [16]:
print("Dataset Shape before deduplicating:",merged_data.shape)
print("Dataset Shape after  deduplicating:",merged_data.drop_duplicates().shape)

Dataset Shape before deduplicating: (829262, 20)
Dataset Shape after  deduplicating: (829262, 20)


In [7]:
merged_data['Revenue'] = merged_data['Units']*merged_data['Actual_Product_Price']
merged_data['xRevenue'] = merged_data['Units']*merged_data['Product_Price']
merged_data['Profit'] = merged_data['Units']*(merged_data['Actual_Product_Price'] - merged_data['Product_Cost'])
print(merged_data.shape)
merged_data.head()

(829262, 20)


,Sale_ID,Date,Store_ID,Product_ID,Units,Year,Quarter,Product_Name,Product_Category,Product_Cost,Product_Price,Month,Actual_Product_Price,Store_Name,Store_City,Store_Location,Store_Open_Date,Revenue,xRevenue,Profit
0,1,2022-01-01,24,4,1,2022,1,Chutes & Ladders,Games,9.99,12.99,1,12.39,Maven Toys Aguascalientes 1,Aguascalientes,Downtown,2010-07-31,12.39,12.99,2.40
1,2,2022-01-01,28,1,1,2022,1,Action Figure,Toys,9.99,15.99,1,15.83,Maven Toys Puebla 2,Puebla,Downtown,2011-04-01,15.83,15.99,5.84
2,3,2022-01-01,6,8,1,2022,1,Deck Of Cards,Games,3.99,6.99,1,6.74,Maven Toys Mexicali 1,Mexicali,Commercial,2003-12-13,6.74,6.99,2.75
3,4,2022-01-01,48,7,1,2022,1,Dart Gun,Sports & Outdoors,11.99,15.99,1,14.97,Maven Toys Saltillo 2,Saltillo,Commercial,2016-03-23,14.97,15.99,2.98
4,5,2022-01-01,44,18,1,2022,1,Lego Bricks,Toys,34.99,38.99,1,38.73,Maven Toys Puebla 3,Puebla,Residential,2014-12-27,38.73,38.99,3.74


In [15]:
merged_data.to_csv('clean_data/store_item_transaction_data.csv',index=False)

Date-Store-Product Level data

In [22]:
date_store_product_rollup_data = merged_data.groupby(['Date','Store_ID','Product_ID','Product_Name','Product_Category','Product_Cost','Product_Price','Actual_Product_Price','Store_Name','Store_City','Store_Location','Store_Open_Date']).agg({'Units':'sum','Revenue':'sum','xRevenue':'sum','Profit':'sum'}).reset_index()
date_store_product_rollup_data.to_csv('clean_data/date_store_product_rollup_data.csv',index=False)

Yr-Month-Store-Product Level data

In [30]:
month_yr_store_product_rollup_data = merged_data.groupby(['Year','Month','Store_ID','Product_ID','Product_Name','Product_Category','Product_Cost','Product_Price','Store_Name','Store_City','Store_Location','Store_Open_Date']).agg({'Units':'sum','Actual_Product_Price':'mean','Revenue':'sum','xRevenue':'sum','Profit':'sum'}).reset_index()
month_yr_store_product_rollup_data.to_csv('clean_data/month_yr_store_product_rollup_data.csv',index=False)
month_yr_store_product_rollup_data

,Year,Month,Store_ID,Product_ID,Product_Name,Product_Category,Product_Cost,Product_Price,Store_Name,Store_City,Store_Location,Store_Open_Date,Units,Actual_Product_Price,Revenue,xRevenue,Profit
0,2022,1,1,1,Action Figure,Toys,9.99,15.99,Maven Toys Guadalajara 1,Guadalajara,Residential,1992-09-18,63,15.517358,978.12,1007.37,348.75
1,2022,1,1,2,Animal Figures,Toys,9.99,12.99,Maven Toys Guadalajara 1,Guadalajara,Residential,1992-09-18,7,12.400000,86.80,90.93,16.87
2,2022,1,1,4,Chutes & Ladders,Games,9.99,12.99,Maven Toys Guadalajara 1,Guadalajara,Residential,1992-09-18,5,12.980000,64.90,64.95,14.95
3,2022,1,1,5,Classic Dominoes,Games,7.99,9.99,Maven Toys Guadalajara 1,Guadalajara,Residential,1992-09-18,11,9.424545,103.67,109.89,15.78
4,2022,1,1,6,Colorbuds,Electronics,6.99,14.99,Maven Toys Guadalajara 1,Guadalajara,Residential,1992-09-18,5,14.980000,74.90,74.95,39.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22926,2023,9,50,30,Rubik's Cube,Games,17.29,19.99,Maven Toys Guanajuato 3,Guanajuato,Residential,2016-05-18,10,19.332000,193.32,199.90,20.42
22927,2023,9,50,31,Splash Balls,Sports & Outdoors,7.99,9.49,Maven Toys Guanajuato 3,Guanajuato,Residential,2016-05-18,50,9.173158,468.48,474.50,68.98
22928,2023,9,50,32,Supersoaker Water Gun,Sports & Outdoors,11.99,14.99,Maven Toys Guanajuato 3,Guanajuato,Residential,2016-05-18,15,13.700000,205.50,224.85,25.65
22929,2023,9,50,34,Toy Robot,Electronics,21.99,27.49,Maven Toys Guanajuato 3,Guanajuato,Residential,2016-05-18,5,27.460000,137.30,137.45,27.35


Date-Product Level data

In [ ]:
date_product_rollup_data = merged_data.groupby(['Date','Product_ID','Product_Name','Product_Category','Product_Cost','Product_Price']).agg({'Units':'sum','Actual_Product_Price':'mean','Revenue':'sum','xRevenue':'sum','Profit':'sum'}).reset_index()
date_product_rollup_data.to_csv('clean_data/date_product_rollup_data.csv',index=False)
date_product_rollup_data

Month-Year-Product Level data

In [28]:
month_yr_product_rollup_data = merged_data.groupby(['Year','Month','Product_ID','Product_Name','Product_Category','Product_Cost','Product_Price']).agg({'Units':'sum','Actual_Product_Price':'mean','Revenue':'sum','xRevenue':'sum','Profit':'sum'}).reset_index()
month_yr_product_rollup_data.to_csv('clean_data/month_yr_product_rollup_data.csv',index=False)
month_yr_product_rollup_data

,Year,Month,Product_ID,Product_Name,Product_Category,Product_Cost,Product_Price,Units,Actual_Product_Price,Revenue,xRevenue,Profit
0,2022,1,1,Action Figure,Toys,9.99,15.99,3441,15.676964,53959.52,55021.59,19583.93
1,2022,1,2,Animal Figures,Toys,9.99,12.99,1513,12.720190,19263.69,19653.87,4148.82
2,2022,1,3,Barrel O' Slime,Art & Crafts,1.99,3.99,76,3.850000,292.60,303.24,141.36
3,2022,1,4,Chutes & Ladders,Games,9.99,12.99,166,12.552112,2083.94,2156.34,425.60
4,2022,1,5,Classic Dominoes,Games,7.99,9.99,276,9.681775,2672.17,2757.24,466.93
...,...,...,...,...,...,...,...,...,...,...,...,...
644,2023,9,31,Splash Balls,Sports & Outdoors,7.99,9.49,1807,9.175609,16554.11,17148.43,2116.18
645,2023,9,32,Supersoaker Water Gun,Sports & Outdoors,11.99,14.99,252,14.531116,3668.85,3777.48,647.37
646,2023,9,33,Teddy Bear,Toys,10.99,12.99,248,12.581613,3131.35,3221.52,405.83
647,2023,9,34,Toy Robot,Electronics,21.99,27.49,486,26.550677,12905.70,13360.14,2218.56


Month-Year-Store Level data

In [36]:
month_yr_store_rollup_data = merged_data.groupby(['Year','Month','Store_ID','Store_Name','Store_City','Store_Location','Store_Open_Date']).agg({'Units':'sum','Product_ID':'nunique','Actual_Product_Price':'mean','Revenue':'sum','xRevenue':'sum','Profit':'sum'}).reset_index()
month_yr_store_rollup_data.rename(columns={'Product_ID':'nUnique_Products'},inplace=True)
month_yr_store_rollup_data.to_csv('clean_data/month_yr_store_rollup_data.csv',index=False)
month_yr_store_rollup_data

,Year,Month,Store_ID,Store_Name,Store_City,Store_Location,Store_Open_Date,Units,nUnique_Products,Actual_Product_Price,Revenue,xRevenue,Profit
0,2022,1,1,Maven Toys Guadalajara 1,Guadalajara,Residential,1992-09-18,563,20,14.612684,8352.78,8546.87,1694.91
1,2022,1,2,Maven Toys Monterrey 1,Monterrey,Residential,1995-04-27,854,16,13.763446,11814.60,12083.46,3521.64
2,2022,1,3,Maven Toys Guadalajara 2,Guadalajara,Commercial,1999-12-27,993,16,13.863232,13526.77,13871.57,3960.20
3,2022,1,4,Maven Toys Saltillo 1,Saltillo,Downtown,2000-01-01,959,17,13.886959,13629.73,13889.41,4284.32
4,2022,1,5,Maven Toys La Paz 1,La Paz,Downtown,2001-05-31,710,18,14.498741,9735.14,10012.40,2783.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,2023,9,46,Maven Toys Guadalajara 4,Guadalajara,Downtown,2015-10-31,949,22,13.236636,12521.81,12880.81,4062.20
1046,2023,9,47,Maven Toys Monterrey 4,Monterrey,Commercial,2015-11-21,1277,27,13.246549,16329.60,16954.33,4324.37
1047,2023,9,48,Maven Toys Saltillo 2,Saltillo,Commercial,2016-03-23,1179,28,11.880530,13090.37,13444.11,4500.16
1048,2023,9,49,Maven Toys Culiacan 1,Culiacan,Downtown,2016-05-10,1186,28,11.812618,14123.58,14679.84,4174.54


Yr_month_store_location

In [9]:
month_yr_store_loc_product_rollup_data = merged_data.groupby(['Year','Month','Product_ID','Product_Name','Product_Category','Product_Cost','Product_Price','Store_City','Store_Location']).agg({'Units':'sum','Actual_Product_Price':'mean','Revenue':'sum','xRevenue':'sum','Profit':'sum'}).reset_index()
month_yr_store_loc_product_rollup_data.to_csv('clean_data/month_yr_store_loc_product_rollup_data.csv',index=False)
month_yr_store_loc_product_rollup_data.head()

,Year,Month,Product_ID,Product_Name,Product_Category,Product_Cost,Product_Price,Store_City,Store_Location,Units,Actual_Product_Price,Revenue,xRevenue,Profit
0,2022,1,1,Action Figure,Toys,9.99,15.99,Aguascalientes,Downtown,45,15.752889,708.88,719.55,259.33
1,2022,1,1,Action Figure,Toys,9.99,15.99,Campeche,Commercial,20,15.861000,317.22,319.80,117.42
2,2022,1,1,Action Figure,Toys,9.99,15.99,Campeche,Downtown,79,15.326835,1210.82,1263.21,421.61
3,2022,1,1,Action Figure,Toys,9.99,15.99,Chetumal,Downtown,116,15.676810,1818.51,1854.84,659.67
4,2022,1,1,Action Figure,Toys,9.99,15.99,Chihuahua,Commercial,18,15.651667,281.73,287.82,101.91


In [10]:
month_yr_store_city_product_rollup_data = merged_data.groupby(['Year','Month','Product_ID','Product_Name','Product_Category','Product_Cost','Product_Price','Store_City']).agg({'Units':'sum','Actual_Product_Price':'mean','Revenue':'sum','xRevenue':'sum','Profit':'sum'}).reset_index()
month_yr_store_city_product_rollup_data.to_csv('clean_data/month_yr_store_city_product_rollup_data.csv',index=False)
month_yr_store_city_product_rollup_data.head()

,Year,Month,Product_ID,Product_Name,Product_Category,Product_Cost,Product_Price,Store_City,Units,Actual_Product_Price,Revenue,xRevenue,Profit
0,2022,1,1,Action Figure,Toys,9.99,15.99,Aguascalientes,45,15.752889,708.88,719.55,259.33
1,2022,1,1,Action Figure,Toys,9.99,15.99,Campeche,99,15.434747,1528.04,1583.01,539.03
2,2022,1,1,Action Figure,Toys,9.99,15.99,Chetumal,116,15.676810,1818.51,1854.84,659.67
3,2022,1,1,Action Figure,Toys,9.99,15.99,Chihuahua,82,15.694268,1286.93,1311.18,467.75
4,2022,1,1,Action Figure,Toys,9.99,15.99,Chilpancingo,32,15.328750,490.52,511.68,170.84


In [12]:
month_yr_loc_product_rollup_data = merged_data.groupby(['Year','Month','Product_ID','Product_Name','Product_Category','Store_Location']).agg({'Units':'sum','Actual_Product_Price':'mean','Product_Cost':'max','Product_Price':'max','Revenue':'sum','xRevenue':'sum','Profit':'sum'}).reset_index()
month_yr_loc_product_rollup_data.to_csv('clean_data/month_yr_loc_product_rollup_data.csv',index=False)
month_yr_loc_product_rollup_data.head()

,Year,Month,Product_ID,Product_Name,Product_Category,Store_Location,Units,Actual_Product_Price,Product_Cost,Product_Price,Revenue,xRevenue,Profit
0,2022,1,1,Action Figure,Toys,Airport,123,15.736549,9.99,15.99,1935.29,1966.77,706.52
1,2022,1,1,Action Figure,Toys,Commercial,631,15.674132,9.99,15.99,9881.61,10089.69,3577.92
2,2022,1,1,Action Figure,Toys,Downtown,2183,15.683211,9.99,15.99,34265.88,34906.17,12457.71
3,2022,1,1,Action Figure,Toys,Residential,504,15.640043,9.99,15.99,7876.74,8058.96,2841.78
4,2022,1,2,Animal Figures,Toys,Airport,116,12.838977,9.99,12.99,1492.71,1506.84,333.87
